# Distances

In [1]:
from ipynb.fs.full.koselleck import *

## Words

In [2]:
# get_veclib_word_data??

In [3]:
def cdist_word(word,
               period_or_periods=None,ymin=YMIN,ymax=YMAX,ybin=YEARBIN,
               run_or_runs=None,neighbors=None,num_runs=1,num_proc=1,progress=True,
              only_valid_words=True,force=False,cache=True):
    try:
#         periods=period_or_periods if period_or_periods else get_periods_bystep(ymin,ymax,ybin)
#         periods,runs=get_periods_runs(periods,run_or_runs,num_runs=num_runs)
#         # load data
#         #dfdist=get_veclib_word_data(word,progress=progress)
        dfdist=distvecs_word(
            word,
            only_valid_words=only_valid_words,
            reformat=True,
            periods=period_or_periods,
            ymin=ymin,ymax=ymax,ybin=ybin,
            num_runs=num_runs,num_proc=num_proc,
            force=force,cache=cache
        )
        if neighbors: dfdist=dfdist[[c for c in dfdist.columns if c in set(neighbors)]]
        if not len(dfdist) or not len(dfdist.columns): return pd.DataFrame()
        if progress: print(f'Finished running cdist_word({word})')
        return dfdist
    except ValueError as e:
        print('!!',e)
        return pd.DataFrame()

In [4]:
cdist_word('value',ymin=1700,ymax=1900,ybin=40)

[Koselleck] (15:06:01) Finished running cdist_word(value) (+0.1s)


abandon  abandoned  abandoning  abandonment     abate  \
word_ period_   run_                                                           
value 1700-1740 1    -0.086230  -0.138428   -0.149839          NaN  0.170819   
      1740-1780 1    -0.039917  -0.119122   -0.016570          NaN -0.048032   
      1780-1820 1    -0.109178  -0.162987   -0.046152     0.156022 -0.030881   
      1820-1860 1    -0.057088  -0.160947   -0.045953     0.158967 -0.048935   
      1860-1900 1    -0.059515  -0.122277   -0.054839     0.164396 -0.059135   

                        abated  abatement      abbe    abbess     abbey  ...  \
word_ period_   run_                                                     ...   
value 1700-1740 1     0.023529   0.273425 -0.170056 -0.081310 -0.056804  ...   
      1740-1780 1    -0.042937   0.343433 -0.062071 -0.128775 -0.009468  ...   
      1780-1820 1    -0.090149   0.279360 -0.008384 -0.136364 -0.058694  ...   
      1820-1860 1    -0.121453   0.310725 -0.049533 -0.099152 -0.033025  ...   
      1860-1900 1    -0.079013   0.173327 -0.025102 -0.113059 -0.055590  ...   

                       zealous    zenith    zephyr      zest      zinc  \
word_ period_   run_                                                     
value 1700-1740 1    -0.106801 -0.073146 -0.201663  0.049158 -0.068101   
      1740-1780 1    -0.145806  0.130629 -0.123099  0.082532  0.036154   
      1780-1820 1    -0.167150  0.095811 -0.097078  0.183338  0.275451   
      1820-1860 1    -0.065448  0.107942 -0.187582  0.215527  0.232066   
      1860-1900 1    -0.057860  0.082108 -0.189007  0.214668  0.145815   

                          zone       zoo  zoological   zoology      zulu  
word_ period_   run_                                                      
value 1700-1740 1    -0.033215  0.354014         NaN       NaN       NaN  
      1740-1780 1    -0.011095  0.274758   -0.006733 -0.000437       NaN  
      1780-1820 1    -0.057494  0.307641    0.042349  0.060060 -0.129499  
      1820-1860 1    -0.018840 -0.022769   -0.010076  0.107711 -0.084016  
      1860-1900 1     0.046594  0.051630    0.045951  0.254679 -0.109424  

[5 rows x 15529 columns]

In [5]:
# dfdist=get_veclib_word_data('culture',progress=True,force=True)

In [8]:
cdist_word('virtue',num_proc=4,neighbors=['value','virtues'], ymin=1700,ymax=1900,ybin=40)

Mapping _distvecs_word_period_() [x4]: 100%|██████████| 5/5 [00:00<00:00,  5.57it/s]
[Koselleck] (14:49:23) Finished running cdist_word(virtue) (+14.8s)


value   virtues
word   period    run                    
virtue 1700-1740 1    0.129532  0.538200
       1740-1780 1    0.052949  0.659482
       1780-1820 1    0.087611  0.708563
       1820-1860 1    0.161590  0.721934
       1860-1900 1    0.223198  0.720987

In [7]:
cdist_word('werwerwerer')

[Koselleck] (18:39:31) Loading cdist data for "werwerwerer" (+2.2s)
[Koselleck] (18:39:31) No file found at /home/ryan/github/koselleck/db/wvecs/db.koselleck.werwerwerer.sqlite (+0.0s)


""


In [9]:
def _cdist_(objd):
    try:
        return cdist_word(**objd)
    except ValueError as e:
        print('!!!!',e,'!!!!')
        return pd.DataFrame()
    
def cdist(word_or_words,period_or_periods=None,run_or_runs=None,neighbors=None,num_runs=10,
          num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
    index_cols=['word','neighbor','period','run']
    words=tokenize_fast(word_or_words) if type(word_or_words)==str else list(word_or_words)
    objs=[
        dict(
            word=word,
            period_or_periods=period_or_periods,
            run_or_runs=run_or_runs,
            neighbors=neighbors,
            num_runs=num_runs,
            num_proc=num_proc if len(word_or_words)==1 else 1,
            progress=progress if len(word_or_words)==1 else False
        )
        for word in words
    ]
    o=pmap(
        _cdist_,
        objs,
        num_proc=num_proc,
        progress=False if len(words)<2 else progress,
        desc='Computing cosine distances for words'
    )
    return pd.concat(o) if len(o) else pd.DataFrame()


In [11]:
cdist('virtue,virtues,fwefwefwewef')

Computing cosine distances for words [x1]: 100%|██████████| 3/3 [00:00<00:00,  3.47it/s]


a'       a's        aa       aaa       aad  \
word    period    run                                                     
virtue  1780-1785 1    2.162901  2.326049  2.425738  2.198551  2.091443   
        1720-1725 1    2.262305  2.393198  2.337339  2.217301  1.925527   
                  2    2.280775  2.379018  2.454623  2.301432  2.291592   
                  3    2.349260  2.257687  2.240709  2.109919  2.185930   
                  4    2.440110  2.178210  2.304128  2.174020  2.072909   
...                         ...       ...       ...       ...       ...   
virtues 1955-1960 6    2.284276       NaN  1.886938       NaN       NaN   
                  7    1.985536       NaN  1.576267       NaN       NaN   
                  8    2.068463       NaN  1.713967       NaN       NaN   
                  9    2.172709       NaN  1.734996       NaN       NaN   
                  10   2.184628       NaN  1.910268       NaN       NaN   

                            aal       aam     aaron  aaronson       aas  ...  \
word    period    run                                                    ...   
virtue  1780-1785 1    2.146191       NaN       NaN       NaN  1.750717  ...   
        1720-1725 1    2.149955  2.260944  2.360189       NaN  2.157606  ...   
                  2    2.157009       NaN  2.184957       NaN  1.867736  ...   
                  3    2.247387       NaN       NaN       NaN  1.873399  ...   
                  4    2.069204  2.326053  2.239394       NaN  2.147869  ...   
...                         ...       ...       ...       ...       ...  ...   
virtues 1955-1960 6         NaN       NaN       NaN       NaN       NaN  ...   
                  7         NaN       NaN       NaN       NaN       NaN  ...   
                  8         NaN       NaN       NaN       NaN       NaN  ...   
                  9         NaN       NaN       NaN       NaN       NaN  ...   
                  10        NaN       NaN       NaN       NaN       NaN  ...   

                       zoom  zoroaster  zos        zr        zs        zu  \
word    period    run                                                       
virtue  1780-1785 1     NaN        NaN  NaN  2.131796  2.212931       NaN   
        1720-1725 1     NaN        NaN  NaN  2.480936  2.306062  2.181680   
                  2     NaN        NaN  NaN  2.340634  2.300155  2.254788   
                  3     NaN        NaN  NaN  2.368693  2.388964  2.161367   
                  4     NaN        NaN  NaN  2.420269  2.234603  2.246385   
...                     ...        ...  ...       ...       ...       ...   
virtues 1955-1960 6     NaN        NaN  NaN       NaN       NaN       NaN   
                  7     NaN        NaN  NaN       NaN       NaN       NaN   
                  8     NaN        NaN  NaN       NaN       NaN       NaN   
                  9     NaN        NaN  NaN       NaN       NaN       NaN   
                  10    NaN        NaN  NaN       NaN       NaN       NaN   

                       zulu  zulus    zurich  zz  
word    period    run                             
virtue  1780-1785 1     NaN    NaN       NaN NaN  
        1720-1725 1     NaN    NaN       NaN NaN  
                  2     NaN    NaN  1.992242 NaN  
                  3     NaN    NaN  1.854614 NaN  
                  4     NaN    NaN       NaN NaN  
...                     ...    ...       ...  ..  
virtues 1955-1960 6     NaN    NaN  2.197970 NaN  
                  7     NaN    NaN  2.289087 NaN  
                  8     NaN    NaN  2.059180 NaN  
                  9     NaN    NaN  2.485581 NaN  
                  10    NaN    NaN  2.458443 NaN  

[960 rows x 31059 columns]

In [ ]:

# def do_cdist(objd):
#     res=cdist(**objd)
#     if res is None: res=pd.DataFrame()
#     return res

# def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
#     index_cols=['word','neighbor','period','run']
#     argd=dict(
#         word=word,period=period,run=run,prefix=prefix,
#         neighbors=neighbors,max_num=max_num,num_runs=num_runs,
#         num_proc=num_proc,force=force,progress=progress,
#         cache_only=cache_only,cache=cache,vl=vl
#     )
#     odf=None
    
#     if type(word)!=str:
#         objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
#         odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=False))
#         return odf if not cache_only else pd.DataFrame()
    
#     with get_veclib(prefix=word,folders=['wvecs'],autocommit=False) as vl:
#         dfdist=pd.DataFrame(dict(vl.items())).T
        
        
        
#         if period is None:
#             objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':1, 'vl':vl}} for prd in get_default_periods()]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()
#         if run is None:
#             objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1, 'vl':vl}} for run in range(num_runs)]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()

#         # load vecs?
#         qstr=f'{period}_{run:02}'
#         sdist=vl.get(qstr)
#         if sdist is None: return pd.DataFrame()
# #         sdist=sdist[set(sdist.index)-{word}]
# #         sdist=sdist*-1 if sdist.min()<-0.1 else sdist
#         if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
#         wddf=pd.DataFrame()
#         wddf['cdist']=sdist
#         wddf=wddf.rename_axis('neighbor').sort_values('cdist')
#         wddf['word']=word
#         wddf['period']=period
#         wddf['run']=run
#         wddf=wddf.reset_index().set_index(index_cols)
#         return wddf


In [ ]:
# dfvecs=vecs('1780-1785',1)
words=get_valid_words()
# words[:100]

In [ ]:
w=random.choice(words)
w

In [ ]:
# %%timeit
# cdist(w,num_proc=8)

In [ ]:
# %%timeit
# cdist(w,num_proc=1)

In [ ]:
# %%timeit
# cdist(w,num_proc=4)

In [ ]:
swords=random.sample(words[:1000],4)
swords

In [ ]:
# %%timeit
# cdist(swords,num_proc=1)

In [ ]:
# %%timeit
# cdist(swords,num_proc=4)

In [ ]:
# cdist(random.sample(words,100),num_proc=4)

In [ ]:
dfdist=cdist('virtues',num_proc=4)
dfdist

In [ ]:
to_nbr('virtues','1720-1725')

In [ ]:
stop

In [ ]:
# cdist(random.sample(words,100),num_proc=4)

In [ ]:
cdist('virtues')

In [ ]:
# # %%timeit
# for word in tqdm(words[:1000]):
#     cdist(word,'1760-1765',progress=False)

In [ ]:
# %%timeit
# cdist('culture','1760-1765',1)

In [ ]:
# dists=cdist('culture',neighbors=['plant','soil','education'],num_proc=1)
# dists

In [ ]:
# distvecs('1770-1775',1)

## Backup

In [ ]:

# def do_cdist(objd):
#     res=cdist(**objd)
#     if res is None: res=pd.DataFrame()
#     return res

# def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
#     index_cols=['word','neighbor','period','run']
#     argd=dict(
#         word=word,period=period,run=run,prefix=prefix,
#         neighbors=neighbors,max_num=max_num,num_runs=num_runs,
#         num_proc=num_proc,force=force,progress=progress,
#         cache_only=cache_only,cache=cache,vl=vl
#     )
#     odf=None
    
#     if type(word)!=str:
#         objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
#         odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=False))
#         return odf if not cache_only else pd.DataFrame()
    
#     with get_veclib(prefix=word,folders=['wvecs'],autocommit=False) as vl:
#         if period is None:
#             objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':1, 'vl':vl}} for prd in get_default_periods()]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()
#         if run is None:
#             objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1, 'vl':vl}} for run in range(num_runs)]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()

#         # load vecs?
#         qstr=f'{period}_{run:02}'
#         sdist=vl.get(qstr)
#         if sdist is None: return pd.DataFrame()
# #         sdist=sdist[set(sdist.index)-{word}]
# #         sdist=sdist*-1 if sdist.min()<-0.1 else sdist
#         if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
#         wddf=pd.DataFrame()
#         wddf['cdist']=sdist
#         wddf=wddf.rename_axis('neighbor').sort_values('cdist')
#         wddf['word']=word
#         wddf['period']=period
#         wddf['run']=run
#         wddf=wddf.reset_index().set_index(index_cols)
#         return wddf
